Inprocessing refers to mitigating the bias in model's predictions during training.
In-processing techniques have several advantages over pre-processing techniques. They can be more effective at addressing bias that is deeply embedded in the data, and they do not require any modifications to the data collection or preprocessing pipeline

1.  Prejudice Remover : 
Prejudice remover works by adding a regularization term to the model's loss function. This regularization term penalizes the model for making predictions that are inconsistent with the desired fairness criteria.

Given the fact that our baseline model was baised against specific gender and race, using prejudice remover should help us get better fairness

In [1]:
import aif360
import holisticai
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from holisticai.bias.mitigation import PrejudiceRemover

Data Gathering

In [2]:
baseline_preprocessed_df = pd.read_csv("../data/data_preprocessed_baseline.csv",index_col=0)
baseline_preprocessed_df = baseline_preprocessed_df.drop(['id','age_cat'],axis=1)

In [24]:
protected_variables = ["sex", "African-American_race"]
output_variable = ["two_year_recid"]

In [25]:
# Simple preprocessing
y = baseline_preprocessed_df[output_variable]
X = pd.get_dummies(baseline_preprocessed_df.drop(protected_variables + output_variable, axis=1))
group = ["African-American_race"]
group_a = baseline_preprocessed_df["African-American_race"] == 1
group_b = baseline_preprocessed_df["African-American_race"] == 0
data_ = [X, y, group_a, group_b]


In [26]:
# Train test split
dataset = train_test_split(*data_, test_size=0.2, shuffle=True)
train_data = dataset[::2]
test_data = dataset[1::2]

In [27]:
# efficacy metrics from sklearn
from sklearn import metrics

# dictionnary of metrics
metrics_dict={
        "Accuracy": metrics.accuracy_score,
        "Balanced accuracy": metrics.balanced_accuracy_score,
        "Precision": metrics.precision_score,
        "Recall": metrics.recall_score,
        "F1-Score": metrics.f1_score}

# efficacy metrics dataframe helper tool
def metrics_dataframe(y_pred, y_true, metrics_dict=metrics_dict):
    metric_list = [[pf, fn(y_true, y_pred)] for pf, fn in metrics_dict.items()]
    return pd.DataFrame(metric_list, columns=["Metric", "Value"]).set_index("Metric")

In [28]:
# import
from holisticai.bias.mitigation import GridSearchReduction

In [31]:
# data and model
X_train, y_train, group_a_train, group_b_train = train_data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
model_lr = LogisticRegression()

In [30]:
# initialize
gsr = GridSearchReduction(constraints="FalsePositiveRateParity")

# incorporate model in gsr
gsr.transform_estimator(model_lr)

GridSearchReduction(constraints='FalsePositiveRateParity')

In [32]:
# fit with data
gsr.fit(X_train_scaled, y_train, group_a_train, group_b_train)

GridSearchReduction(constraints='FalsePositiveRateParity')

In [48]:
# predict test
X_test, y_test, group_a_test, group_b_test = test_data
X_test_scaled = scaler.transform(X_test)
y_pred = gsr.predict(X_test_scaled)
y_proba = gsr.predict_proba(X_test_scaled)
y_score = y_proba[:,1]
y_true = y_test

In [38]:
# Baseline efficacy
metrics_dataframe(y_pred, y_true)

,Value
Metric,
Accuracy,0.715379
Balanced accuracy,0.706637
Precision,0.703509
Recall,0.625585
F1-Score,0.662263


In [36]:
# bias
# holisticai imports
from holisticai.bias.metrics import classification_bias_metrics
classification_bias_metrics(group_a_test, group_b_test, y_pred, y_true, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,0.220214,0
Disparate Impact,1.783364,1
Four Fifths Rule,0.560738,1
Cohen D,0.461969,0
2SD Rule,8.521605,0
Equality of Opportunity Difference,0.210739,0
False Positive Rate Difference,0.127932,0
Average Odds Difference,0.169336,0
Accuracy Difference,-0.003895,0


<b>prejudice remover

In [43]:
model_lr2 = LogisticRegression()

In [53]:
pr2 = PrejudiceRemover(eta=50)
pr2.transform_estimator()

PrejudiceRemover(eta=50)

In [54]:
pr2.fit( X=X_train_scaled, y_true=y_train, group_a=group_a_train, group_b=group_b_train)

PrejudiceRemover(eta=50)

In [55]:
# predict test
y_pred = pr2.predict(X_test_scaled,group_a=group_a_test,group_b=group_b_test)
y_proba = pr2.predict_proba(X_test_scaled,group_a=group_a_test,group_b=group_b_test)
y_score = y_proba[:,1]
y_true = y_test

In [56]:
# Baseline efficacy
metrics_dataframe(y_pred, y_true)

,Value
Metric,
Accuracy,0.674322
Balanced accuracy,0.663197
Precision,0.658716
Recall,0.560062
F1-Score,0.605396


In [57]:
# bias
# holisticai imports
from holisticai.bias.metrics import classification_bias_metrics
classification_bias_metrics(group_a_test, group_b_test, y_pred, y_true, metric_type='both')

,Value,Reference
Metric,,
Statistical Parity,0.000101,0
Disparate Impact,1.000266,1
Four Fifths Rule,0.999734,1
Cohen D,0.000208,0
2SD Rule,0.003940,0
Equality of Opportunity Difference,-0.054226,0
False Positive Rate Difference,-0.043106,0
Average Odds Difference,-0.048666,0
Accuracy Difference,-0.029127,0
